# Automated Conversion of AutoGluon Models to PyTorch with Quantization-Aware Training

This notebook will automatically analyze the architecture of the model generated by AutoGluon and replicate it to generate an equivalent PyTorch model. It will then apply Quantization-Aware Training (QAT) to the replicated PyTorch model.


Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from autogluon.tabular import TabularDataset, TabularPredictor
import matplotlib.pyplot as plt
import torch.quantization as quant

: 

Load and preprocess the dataset

In [ ]:
predictor_path= './datasets/CICIDS2017/balanced_binary/automl_search/models'  # Modify as needed
# Load the datasets
train_data = pd.read_csv('./datasets/CICIDS2017/balanced_binary/train.csv')
test_data = pd.read_csv('./datasets/CICIDS2017/balanced_binary/test.csv')
val_data = pd.read_csv('./datasets/CICIDS2017/balanced_binary/validation.csv')

# Drop the ID column
train_data = train_data.drop(columns=['ID'])
test_data = test_data.drop(columns=['ID'])
val_data = val_data.drop(columns=['ID'])

# Encode labels
label_encoder = LabelEncoder()
train_data['Label'] = label_encoder.fit_transform(train_data['Label'])
test_data['Label'] = label_encoder.transform(test_data['Label'])
val_data['Label'] = label_encoder.transform(val_data['Label'])

# Convert to TabularDataset
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)
val_data = TabularDataset(val_data)


Load the autogluon model


In [ ]:
# Load original model
original_predictor = TabularPredictor.load(predictor_path)
best_model = original_predictor._trainer.load_model(original_predictor.model_best)

 Extract the architecture of the best AutoGluon model

In [ ]:
# Print the model architecture details
print(f"Model architecture:\n{best_model.model}")

# Extract the architecture
architecture = []
for name, layer in best_model.model.named_children():
    if isinstance(layer, nn.Sequential):
        for sub_layer in layer:
            architecture.append((type(sub_layer), sub_layer))
    else:
        architecture.append((type(layer), layer))

print("Extracted Architecture:")
for layer_type, layer_obj in architecture:
    print(layer_type, layer_obj)

Replicate the architecture in PyTorch automatically

In [ ]:
class AutoReplicatedNN(nn.Module):
    def __init__(self, architecture):
        super(AutoReplicatedNN, self).__init__()
        layers = []
        for layer_type, layer_obj in architecture:
            if layer_type == nn.BatchNorm1d:
                layers.append(nn.BatchNorm1d(layer_obj.num_features))
            elif layer_type == nn.Linear:
                layers.append(nn.Linear(layer_obj.in_features, layer_obj.out_features))
            elif layer_type == nn.ReLU:
                layers.append(nn.ReLU())
            elif layer_type == nn.Dropout:
                layers.append(nn.Dropout(p=layer_obj.p))
            elif layer_type == nn.Softmax:
                layers.append(nn.Softmax(dim=layer_obj.dim))
            else:
                raise ValueError(f"Unhandled layer type: {layer_type}")
        self.main_block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.main_block(x)

# Initialize the replicated model
replicated_model = AutoReplicatedNN(architecture)
print(replicated_model)


Compare AutoGluon and PyTorch model architectures

In [ ]:
# Print the AutoGluon model architecture
print("AutoGluon Model Architecture:")
print(best_model.model)

# Print the PyTorch replicated model architecture
print("\nReplicated PyTorch Model Architecture:")
print(replicated_model)

Train the replicated PyTorch model



In [ ]:
# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(replicated_model.parameters(), lr=0.001)

# Training parameters
num_epochs = 20
batch_size = 64

# Convert training data to tensors
X_train = train_data.drop(columns=['Label']).values
y_train = train_data['Label'].values
X_val = val_data.drop(columns=['Label']).values
y_val = val_data['Label'].values

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Train the replicated model
for epoch in range(num_epochs):
    permutation = torch.randperm(X_train_tensor.size()[0])
    for i in range(0, X_train_tensor.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_X, batch_y = X_train_tensor[indices], y_train_tensor[indices]
        
        # Forward pass
        outputs = replicated_model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validate the model
    replicated_model.eval()
    with torch.no_grad():
        val_outputs = replicated_model(X_val_tensor)
        _, val_predicted = torch.max(val_outputs.data, 1)
        val_accuracy = accuracy_score(y_val_tensor, val_predicted)
    replicated_model.train()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.4f}')

Evaluate the replicated PyTorch model


In [ ]:
# Convert test data to tensors
X_test = test_data.drop(columns=['Label']).values
y_test = test_data['Label'].values

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Evaluate the replicated model on the test set
replicated_model.eval()
with torch.no_grad():
    outputs = replicated_model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    replicated_accuracy = accuracy_score(y_test_tensor, predicted)
    print(f'Accuracy on the test set (Replicated PyTorch Model): {replicated_accuracy:.4f}')

Apply Quantization-Aware Training (QAT)

In [ ]:
from torch.quantization import QuantStub, DeQuantStub

class QATWrapper(nn.Module):
    def __init__(self, model):
        super(QATWrapper, self).__init__()
        self.model = model
        self.quant = QuantStub()
        self.dequant = DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        x = self.model(x)
        x = self.dequant(x)
        return x

qat_model = QATWrapper(replicated_model)

# Prepare the model for QAT
qat_model.qconfig = torch.quantization.get_default_qat_qconfig('qnnpack')
torch.quantization.prepare_qat(qat_model, inplace=True)

# Fine-tune the model with QAT
for epoch in range(num_epochs):
    permutation = torch.randperm(X_train_tensor.size()[0])
    for i in range(0, X_train_tensor.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch_X, batch_y = X_train_tensor[indices], y_train_tensor[indices]
        
        # Forward pass
        outputs = qat_model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validate the model
    qat_model.eval()
    with torch.no_grad():
        val_outputs = qat_model(X_val_tensor)
        _, val_predicted = torch.max(val_outputs.data, 1)
        val_accuracy = accuracy_score(y_val_tensor, val_predicted)
    qat_model.train()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Convert to a quantized model
quantized_model_qat = torch.quantization.convert(qat_model.eval(), inplace=False)


 Evaluate the quantized QAT PyTorch model

In [ ]:
# Evaluate the quantized model on the test set
quantized_model_qat.eval()
with torch.no_grad():
    outputs = quantized_model_qat(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    quantized_qat_accuracy = accuracy_score(y_test_tensor, predicted)
    print(f'Accuracy on the test set after quantization-aware training: {quantized_qat_accuracy:.4f}')

Compare results and plot

In [ ]:
# Evaluate the original AutoGluon model
original_ag_model_accuracy = original_predictor.evaluate(test_data)['accuracy']

# Prepare data for plotting
models = ['Original AutoGluon Model', 'Replicated PyTorch Model', 'Quantized QAT PyTorch Model']
accuracies = [original_ag_model_accuracy, replicated_accuracy, quantized_qat_accuracy]

# Plot the results
plt.figure(figsize=(10, 6))
plt.bar(models, accuracies, color=['blue', 'orange', 'green'])
plt.ylim(0, 1)
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison of AutoGluon and PyTorch Models')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()